In [61]:
# Imports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras 
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dropout
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder 
from sklearn.metrics import f1_score

In [62]:
# Read Input
rows_train = 112000 # Max 112'000
train = pd.read_csv('train.csv', nrows = rows_train)

In [63]:
# Map letter to category

def map_to_cat(letter):
  if letter == 'R' or letter == 'H' or letter == 'K': # AA with positively charged side chains
    return 'B'
  elif letter == 'D' or letter == 'E': # AA with negatively charged side chains
    return 'J'
  elif letter == 'S' or letter == 'T' or letter == 'N' or letter == 'Q': # AA with polar uncharged side chains
    return 'O'
  elif letter == 'C' or letter == 'U' or letter == 'G' or letter == 'P': # special cases
    return 'X'
  else: # AA with hydrophobic side chains
    return 'Z'

In [64]:
alphabet    = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
train_tf    = train.values
X_train     = train_tf[:,0]
y_train     = train_tf[:,1]
X_train_int = np.zeros((rows_train, 8))

for i in range(rows_train):
  letters = X_train[i]
  letters = letters + map_to_cat(letters[0]) + map_to_cat(letters[1]) + map_to_cat(letters[2]) + map_to_cat(letters[3])
  X_train_int[i,:]= [char_to_int[char] for char in letters]

one_hot_train = tf.one_hot(X_train_int, 26, dtype=tf.uint8)


In [65]:
X_train_np = np.array(one_hot_train)
X_train_np.resize(rows_train,208)
X_train_pd = pd.DataFrame(X_train_np)

In [70]:
from sklearn.model_selection import train_test_split

x_training, x_valid, y_training, y_valid = train_test_split(X_train_np, y_train, test_size=0.01)

In [71]:
model = Sequential()
model.add(Dense(208, input_dim = 208, activation = 'relu'))
model.add(Dropout(0.2, input_shape=(208,)))
model.add(Dense(208, activation = 'relu'))
model.add(Dropout(0.2, input_shape=(208,)))
model.add(Dense(52, activation = 'relu'))
model.add(Dropout(0.2, input_shape=(52,)))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_training, y_training, epochs=50, batch_size=500)

Epoch 1/50
110880/110880 [==============================] - 2s 19us/step - loss: 0.0947 - accuracy: 0.9665
Epoch 2/50
110880/110880 [==============================] - 2s 15us/step - loss: 0.0448 - accuracy: 0.9823
Epoch 3/50
110880/110880 [==============================] - 2s 17us/step - loss: 0.0321 - accuracy: 0.9873
Epoch 4/50
110880/110880 [==============================] - 2s 14us/step - loss: 0.0263 - accuracy: 0.9897
Epoch 5/50
110880/110880 [==============================] - 2s 15us/step - loss: 0.0223 - accuracy: 0.9912
Epoch 6/50
110880/110880 [==============================] - 2s 15us/step - loss: 0.0195 - accuracy: 0.9923
Epoch 7/50
110880/110880 [==============================] - 2s 15us/step - loss: 0.0176 - accuracy: 0.9932
Epoch 8/50
110880/110880 [==============================] - 2s 17us/step - loss: 0.0159 - accuracy: 0.9937
Epoch 9/50
110880/110880 [==============================] - 2s 16us/step - loss: 0.0141 - accuracy: 0.9943
Epoch 10/50
110880/110880 [==========

In [72]:
model_predict = model.predict_classes(x_valid)
y_predicted = pd.DataFrame(model_predict, dtype=int)
y_true = pd.DataFrame(y_valid,dtype=int)
score = f1_score(y_predicted, y_true)
score

0.9156626506024096

Submission

In [73]:
rows_test = 48000 # Max 48'000
test = pd.read_csv('test.csv', nrows = rows_test)
char_to_int = dict((c, i) for i, c in enumerate('ABCDEFGHIJKLMNOPQRSTUVWXYZ'))
test_tf    = test.values
X_test      = test_tf[:,0]
X_test_int = np.zeros((48000, 8))

for i in range(48000):
  letters = X_test[i]
  letters = letters + map_to_cat(letters[0]) + map_to_cat(letters[1]) + map_to_cat(letters[2]) + map_to_cat(letters[3])
  X_test_int[i,:]= [char_to_int[char] for char in letters]

one_hot_test = tf.one_hot(X_test_int, 26, dtype=tf.uint8)
X_test_np = np.array(one_hot_test)
X_test_np.resize(48000,208)
X_test_pd = pd.DataFrame(X_test_np)

In [74]:
prediction = pd.DataFrame(model.predict_classes(X_test_pd))
prediction.round(0)
np.savetxt("prediction.csv", prediction, fmt="%d")